# Dependencies

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.optimize import minimize

import csv
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D

import matplotlib as mpl
import importlib
importlib.reload(mpl); importlib.reload(plt); importlib.reload(sns)

import glob
import os
from IPython.display import clear_output

# Functions for RMSD and CC calculation

In [2]:
def RMSD_btw(ygrid_A, ygrid_B): # Data has to be interpolated if one of the spectra was shifted!!!
    
    array_A = np.array(ygrid_A)
    array_B = np.array(ygrid_B)
    
    key = ~np.isnan(array_A+array_B) # remove all NaNs

    array_A = array_A[key]
    array_B = array_B[key]

    return np.sqrt(np.sum((array_A-array_B)**2)/np.sum(array_B**2))


def CC_btw(ygrid_A, ygrid_B): # Data has to be interpolated if one of the spectra was shifted!!!
      
    array_A = np.array(ygrid_A)
    array_B = np.array(ygrid_B)
    
    key = ~np.isnan(array_A+array_B) # remove all NaNs

    array_A = array_A[key]
    array_B = array_B[key]
    
    mean_A = np.mean(array_A)
    mean_B = np.mean(array_B)
    
    return np.sum((array_A-mean_A)*(array_B-mean_B))/np.sqrt(np.sum((array_A-mean_A)**2)*np.sum((array_B-mean_B)**2))

# Hyperparameters

In [5]:
#Names of the columns in data files
colnames=['Frequency', 'Impedance', 'Phase', 'Real', 'Imaginary', 'Magnitude'] 

# Name of parameter and result files
parameter_path = r'data\trainable_parameters.dat'
result_path = r'data\result.dat'

In [4]:
#Name of the columns used for analysis
X_NAME = 'Frequency'
Y_NAME = 'Impedance'

#Frequency range
FREQ_MIN = 43600 #Minimum frequency in Hz
FREQ_MAX = 79800 #Maximum frequency in Hz

#Used methods for ...
MIN_METHOD = 'Nelder-Mead' # Minimization
INT_METHOD = 'linear' # Interpolation

# Form of horizontal shift (in this case: linear)
def horizontal_shift(x, p): 
    (a, b) = p # Parameters
    
    x_new = x+(a+b*x) # Formula
     
    return x_new

hor_PARAM_INIT = np.array([0, 0]) # initial parameters for regression
hor_PARAM_RANGE =  ((-3000, 3000), (-0.01, 0.01)) # parameter ranges for regression

# Form of vertical shift (in this case: cubic)
def vertical_shift(y, p, T): 
    (a, b, c, d) = p # Parameters
    
    y_new = y+a+b*T+c*T**2+d*T**3 # Formula
    
    return y_new

ver_DIM = 4 # Number of Parameters for vertical shift (in this case: a, b, c and d)